In [1]:
import nltk
nltk.download('words', quiet=True)
nltk.download('names', quiet=True)

True

In [2]:
import urllib.request

# Load the CMU Pronouncing Dictionary from the internet
url = "http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b"
with urllib.request.urlopen(url) as f:
    cmudict = f.read().decode("ISO-8859-1")

In [3]:
import pyphonetics
import pronouncing


# Define a function that generates homophones for a given word
def generate_homophones(word):
    homophones = []
    # Create an instance of the Soundex algorithm
    soundex = pyphonetics.RefinedSoundex() #There are many others but this is the best one
    # Convert the input word to its Soundex code
    try:
        soundex_code = soundex.phonetics(word)

    # Compare the Soundex code of the input word with the Soundex code of each word in the CMU Pronouncing Dictionary
        for line in cmudict.splitlines():
            if line.startswith(";;;"):
                continue
            line = line.strip().split()
            pron = "".join(line[1:])
            pron_code = soundex.phonetics(pron)
            if soundex_code == pron_code:
                homophones.append(line[0])

        # Return the list of homophones
        homophones.extend(pronouncing.rhymes(word))
    except IndexError:
        return pronouncing.rhymes(word)
    return list(set(homophones))

In [4]:
# Testing our thingy, works pretty good but has bizzare words 
word = "adore"

# Generate homophones for the input word using the CMU Pronouncing Dictionary and Pyphonetics, loaded from the nettttt
homophones = generate_homophones(word)

# Liste of homophones
if len(homophones) > 0:
    print(f"Homophones for {word}: {', '.join(homophones)}")
else:
    print(f"No homophones found for {word}.")


Homophones for adore: corr, livermore, non-core, paramore, roar, anti-war, goar, sore, war, boar, longcor, bore, spore, tor, fore, ngor, anymore, knorr, underscore, ashore, soar, snore, gilgore, mor, clore, loar, nor, roehr, implore, schor, cor, sedor, laur, nohr, lenore, espectador, d'or, boer, spaur, stohr, pour, torr, por, borre, doerr, balthazor, morr, ACERO(2), bensenyore, villasenor, hoerr, abor, ATARI, kohr, yore, elnore, offshore, gorr, lor, ADOREE, moore, door, labore, delore, gore, horr, drawer, forr, jambor, lahore, lalor, igor, chore, hoar, hoare, ADORA, faure, whore, schnorr, bohr, decor, guarantor, thor, dohr, livor, orr, lamaur, saur, score, glore, restore, oar, shor, wor, schorr, cavalli-sfor, stoehr, noncore, baur, rumore, heretofore, laure, wore, inshore, loehr, vore, lazor, devor, floor, more, pore, ohr, shorr, four, outpour, wherefore, galore, dior, coar, or, flor, rapport, deplore, swore, glor, sor, senor, cohr, mazor, store, sotomayor, shore, ador, ignore, montefi

In [4]:

def is_english_word_or_name(word): #APIS sometimes return with names and soetimes return just false words

    # Download the English names corpus if necessary
    nltk.download('names', quiet=True)
    
    # Get a set of English words and names
    english_words = set(nltk.corpus.words.words())
    english_names = set(nltk.corpus.names.words())
    
    # Check if the word is in the set of English words or names
    if word.lower() in english_words or word.title() in english_names:
        return True
    else:
        return False


In [5]:
import re

def clean_words(words): # Some word have special characters

    clean_words = []
    for word in words:
        clean_word = re.sub(r'[^a-zA-Z]', '', word)
        if clean_word and is_english_word_or_name(word):
            clean_words.append(clean_word)
    return [w.upper() for w in clean_words]


In [9]:
clean_words(homophones) # just testing

NameError: name 'homophones' is not defined

In [6]:

import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet as wn
from scipy.spatial.distance import cosine, euclidean, cityblock, jaccard

# Load un corpus pour les mots
nltk.download('brown')
corpus = brown.words()

# creer une distribution
fdist = nltk.FreqDist(corpus)
def dist_sim(word1, word2, metric='euclidean'): # Similarity to choose the most textually proche word
    # get la freq du corpus
    freq1 = fdist[word1.lower()]
    freq2 = fdist[word2.lower()]

    # get tous les mots du corpus
    vocab = set(corpus)

    # rep vect des mots
    vec1 = [freq1 if w == word1 else 0 for w in vocab]
    vec2 = [freq2 if w == word2 else 0 for w in vocab]

    # Calculate similarity
    if metric == 'cosine':
        sim = 1 - cosine(vec1, vec2)
    elif metric == 'euclidean':
        sim = 1 / (1 + euclidean(vec1, vec2))
    elif metric == 'manhattan':
        sim = 1 / (1 + cityblock(vec1, vec2))
    elif metric == 'jaccard':
        sim = 1 / (1 + jaccard(vec1, vec2))
    else:
        raise ValueError('Unsupported distance metric')

    return sim




[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [7]:
import numpy as np

def replace_word(input_word, word_list): # Choosing most similar word to replace with from list
  
    similarity_scores=[]
    
    for w in word_list: # we calculate textual similarity between each word in the list and our word 
        similarity_scores.append(dist_sim(input_word,w)) 
    
    if len(similarity_scores)==0:
        return None
    # Get the index of the word in word_list with the highest cosine similarity to input_word
    most_similar_word_index = np.argmax(similarity_scores)
    
    # Get the most similar word from word_list
    most_similar_word = word_list[most_similar_word_index]
    
    return most_similar_word


In [8]:
import random

def replace_word(word, word_list):
    if not word_list:
        return word
    else:
        return random.choice(sorted(word_list))


In [9]:
import random
def put_word(sentence, word, potential_replacement):
    if potential_replacement is None or word is None:
        return sentence
    else:
        return sentence.replace(" " + word + " ", " " + potential_replacement + " ")

def generate_dataset(df, column_name):
    
    df["sentence 1"] = df[column_name]
    df["sentence 2"] = df[column_name]
    for i in range(len(df)):
        if i<=2369: continue
        sentence_1 = df.loc[i, "sentence 1"]
        sentence_2 = df.loc[i, "sentence 2"]
        words_1 = sentence_1.split()
        words_2 = sentence_2.split()
        n = random.randint(0, len(words_1))
        m = random.randint(0, len(words_2) - n)
        indices_1 = random.sample(range(len(words_1)), n)
        indices_2 = random.sample([i for i in range(len(words_2)) if i not in indices_1], m)
        for j in indices_1:
            potential_replacement = replace_word(words_1[j],list(set(clean_words(generate_homophones(words_1[j])))))
            sentence_1 = put_word(sentence_1, words_1[j], potential_replacement)
        for j in indices_2:
            potential_replacement = replace_word(words_2[j],list(set(clean_words(generate_homophones(words_2[j])))))
            sentence_2 = put_word(sentence_2, words_2[j], potential_replacement)
        df.loc[i, "sentence 1"] = sentence_1
        df.loc[i, "sentence 2"] = sentence_2
        print(f"Iteration number {i}")
        df.to_excel('synthetised-dataset.xlsx')
        
    return df[["sentence 1", "sentence 2", column_name]]

In [10]:
import pandas as pd
data = pd.read_excel('dataset.xlsx')
data.drop(data.columns[0], axis=1, inplace=True)
datadf = generate_dataset(data,data.columns[0])
datadf.to_excel("synthetised-dataset.xlsx")

Iteration number 2370
Iteration number 2371
Iteration number 2372
Iteration number 2373
Iteration number 2374
Iteration number 2375
Iteration number 2376
Iteration number 2377
Iteration number 2378
Iteration number 2379
Iteration number 2380
Iteration number 2381
Iteration number 2382
Iteration number 2383
Iteration number 2384
Iteration number 2385
Iteration number 2386
Iteration number 2387
Iteration number 2388
Iteration number 2389
Iteration number 2390
Iteration number 2391
Iteration number 2392
Iteration number 2393
Iteration number 2394
Iteration number 2395
Iteration number 2396
Iteration number 2397
Iteration number 2398
Iteration number 2399
Iteration number 2400
Iteration number 2401
Iteration number 2402
Iteration number 2403
Iteration number 2404
Iteration number 2405
Iteration number 2406
Iteration number 2407
Iteration number 2408
Iteration number 2409
Iteration number 2410
Iteration number 2411
Iteration number 2412
Iteration number 2413
Iteration number 2414
Iteration 

[nltk_data] Error loading names: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


Iteration number 2442
Iteration number 2443
Iteration number 2444
Iteration number 2445
Iteration number 2446
Iteration number 2447
Iteration number 2448
Iteration number 2449
Iteration number 2450
Iteration number 2451
Iteration number 2452
Iteration number 2453
Iteration number 2454
Iteration number 2455
Iteration number 2456
Iteration number 2457
Iteration number 2458
Iteration number 2459
Iteration number 2460
Iteration number 2461
Iteration number 2462
Iteration number 2463
Iteration number 2464
Iteration number 2465
Iteration number 2466
Iteration number 2467
Iteration number 2468
Iteration number 2469
Iteration number 2470
Iteration number 2471
Iteration number 2472
Iteration number 2473
Iteration number 2474
Iteration number 2475
Iteration number 2476
Iteration number 2477
Iteration number 2478
Iteration number 2479
Iteration number 2480
Iteration number 2481
Iteration number 2482
Iteration number 2483
Iteration number 2484
Iteration number 2485
Iteration number 2486
Iteration 

KeyboardInterrupt: 

# Approche 4: Amelioration de l'approche 1
## Steps :
### Target is just the copy of the correct sentence :
### Sentence 1 is a copy of the target with random homophones on randon spots
### Sentence 2 is the Target, first decomposed to morphemes, THEN replaced with homphones
#### Disclamer : A morpheme example : morpheme(unhappy)--> un-happy
#### Utility : Evaluating the case where we have bi-grams


In [ ]:
import nltk

# Download the required nltk modules
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Function to segment a sentence into morphemes
def segment_sentence(sentence):
    # Tokenize 
    words = nltk.word_tokenize(sentence)
    
    # POS TAGGING to get verbs et d'autres stuff
    tagged_words = nltk.pos_tag(words)
    
    # Segmentation using WordNet
    morphemes = []
    for word, tag in tagged_words:
        if tag.startswith('NN'):  # Nouns
            segments = nltk.corpus.wordnet.morphy(word, nltk.corpus.wordnet.NOUN)
        elif tag.startswith('VB'):  # Verbs
            segments = nltk.corpus.wordnet.morphy(word, nltk.corpus.wordnet.VERB)
        elif tag.startswith('JJ'):  # Adjectives
            segments = nltk.corpus.wordnet.morphy(word, nltk.corpus.wordnet.ADJ)
        elif tag.startswith('RB'):  # Adverbs
            segments = nltk.corpus.wordnet.morphy(word, nltk.corpus.wordnet.ADV)
        else:
            segments = [word]  # Si on n'a pas un tag on garde le mot tel qu'il est
        
        # Nzidou the morpheme segments 
        if segments is not None:  # check if segments is not None
        # Nzidou the morpheme segments 
            morphemes.extend(segments)
    
    # Return the list of morphemes as a string parce qu'on a besoin de la phrase
    return ' '.join(morphemes)

In [ ]:
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')


# Load the sentences from the local file
with open('wikisent2.txt', 'r', encoding='utf-8') as f:
    sentences = f.read().splitlines()

# Create a pandas dataframe with three columns
data = pd.DataFrame(columns=['sentence1', 'sentence2', 'target'])

# Iterate through the sentences and create new rows in the dataframe
for i, sentence in enumerate(sentences):
    # Create a new row with the original sentence in all three columns
    row = {'sentence1': sentence, 'sentence2': sentence, 'target': sentence}
    
    # Split the sentence into words
    words1 = sentence.split()
    words2 = segment_sentence(sentence).split()
    print(f"Iteration number {i}")
    
    # Iterate through the words and replace them randomly in either sentence1 or sentence2
    for i, word in enumerate(words1):
        if re.match("^[a-zA-Z]*$", word):
            potential_replacement = replace_word(word,list(set(clean_words(generate_homophones(word)))))
            if potential_replacement is not None:
                
                if i % 2 == 0:
                    row['sentence1'] = row['sentence1'].replace(word, potential_replacement)
                else:
                    continue
            else:
                row['sentence1'] = row['sentence1']
                
        else:
            row['sentence1'] = row['sentence1']
            
        data.to_csv('dataset.csv')
        
    for i, word in enumerate(words2):
        if re.match("^[a-zA-Z]*$", word):
            potential_replacement = replace_word(word,list(set(clean_words(generate_homophones(word)))))
            if potential_replacement is not None:
                
                if i % 2 == 0:
                    continue
                else:
                    row['sentence2'] = row['sentence2'].replace(word, potential_replacement)
            else:
                row['sentence2'] = row['sentence2']
        else:
            row['sentence2'] = row['sentence2']
        data.to_csv('dataset-morph.csv')
    
    # Append the new row to the dataframe
    data = data.append(row, ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: 'wikisent2.txt'